9. Суммируя весь предыдущий опыт напишите программу, которая будет скрейпить данные из imdb. Рекомендуемые библиотеки: BeautifulSoup, requests и random. Программа должна выполнять следующий функционал:

    a. При запуске программа должна подключаться к странице https://www.imdb.com/chart/top

    b. Собирать список фильмов (и по вашему желанию их описание)

    c. Выводить 10 случайных фильмов

In [18]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [19]:
import requests
from bs4 import BeautifulSoup
import random
from IPython.display import HTML, display

In [ ]:
class Movie:
    def __init__(self, poster_url, title, year, rating, description=''):
        self.poster_url = poster_url
        self.title = title
        self.year = year
        self.rating = rating
        self.description = description
    
    @classmethod
    def from_li_str(cls, li_str):
        soup = BeautifulSoup(li_str, 'lxml')
        poster_url = soup.find('img')['src']
        title_full = soup.find('h3', class_='ipc-title__text').text.strip()
        title = title_full.split('. ', 1)[1] if '. ' in title_full else title_full
        year = soup.find('span', class_='cli-title-metadata-item').text.strip()[:4]
        rating = soup.find('span', class_='ipc-rating-star--imdb').text.strip()[:3]
        return cls(poster_url, title, year, rating)

    def get_dict(self):
        return {'poster': self.poster_url, 
                'title': self.title, 
                'year': self.year, 
                'rating': self.rating, 
                'description': self.description}
    
    def display_movie(self):
        html = f"""
        <div style="display: flex; align-items: center; margin-bottom: 20px;">
            <div style="margin-right: 20px;">
                <img src="{self.poster_url}" alt="{self.title}" style="width: 150px; height: auto;">
            </div>
            <div>
                <h3>{self.title}</h3>
                <p>Год: <strong>{self.year}</strong></p>
                <p>Оценка IMDb: <strong>{self.rating}</strong></p>
            </div>
        </div>
        """
        display(HTML(html))

In [39]:
url = 'https://www.imdb.com/chart/top/'

In [40]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
response.raise_for_status()

In [41]:
soup = BeautifulSoup(response.text, 'lxml')
movies_html = soup.find('ul', role='presentation')

In [42]:
movies = []
for movie_html in movies_html.find_all('li'):
    movies.append(Movie.from_li_str(str(movie_html)))

In [43]:
random_movies = random.sample(movies, 10)
for movie in random_movies:
    movie.display_movie()